## Feature Selection with Mutual Information

The aim of this notebook:
- Look at column features and how they influence a cell's self-exciting property (mutual info)

In [1]:
import os
import logging as log
from time import strftime
from copy import deepcopy
from torch import nn, optim
import torch.nn.functional as F
from utils.data_processing import *
from logger.logger import setup_logging
from utils.configs import BaseConf
from utils.utils import write_json, Timer, get_data_sub_paths, pshape, get_data_resolutions
from models.kangkang_fnn_models import KangFeedForwardNetwork, SimpleKangFNN
from dataloaders.flat_loader import FlatDataLoaders, MockLoader, MockLoaders
from datasets.flat_dataset import FlatDataGroup

from dataloaders.cell_loader import CellDataLoaders
from datasets.cell_dataset import CellDataGroup

from utils.metrics import PRCurvePlotter, ROCCurvePlotter, LossPlotter, PerTimeStepPlotter
from sklearn.metrics import accuracy_score, average_precision_score, roc_auc_score
from models.model_result import ModelResult, ModelMetrics, save_metrics
from utils.mock_data import mock_fnn_data_classification
from utils.plots import im
from trainers.generic_trainer import train_model
from models.kangkang_fnn_models import train_epoch_for_fnn
from utils.configs import BaseConf
from utils.metrics import best_threshold, get_y_pred
from dataloaders.grid_loader import GridDataLoaders
from datasets.grid_dataset import GridDataGroup

from utils.mock_data import mock_fnn_data_classification
import matplotlib.pyplot as plt
from utils.plots import im
from utils.metrics import best_threshold, get_y_pred, get_y_pred_by_thresholds, best_thresholds
from models.model_result import ModelResult, ModelMetrics, save_metrics, compare_all_models,\
                                get_models_metrics, get_models_results
from pprint import pprint
from time import time
from utils.setup import setup

In [20]:
conf = BaseConf()

conf.model_name = f"Mutual Info"

conf.data_path = f"./data/processed/{data_sub_path}/"

if not os.path.exists(conf.data_path):
    raise Exception(f"Directory ({conf.data_path}) needs to exist.")

conf.model_path = f"{conf.data_path}models/{conf.model_name}/"
os.makedirs(conf.data_path, exist_ok=True)
os.makedirs(conf.model_path, exist_ok=True)

setup_logging(save_dir=conf.model_path,
              log_config='./logger/standard_logger_config.json',
              default_level=log.INFO)

with np.load(conf.data_path + "generated_data.npz") as zip_file:  # context helper ensures zip_file is closed
    sparse_crimes = zip_file["crime_types_grids"]
    crime_feature_indices = zip_file["crime_feature_indices"]


shaper = Shaper(data=sparse_crimes,
                conf=conf)

#     return conf, shaper, sparse_crimes, crime_feature_indices


In [2]:
data_sub_paths = get_data_sub_paths()
pprint(np.sort(data_sub_paths))

data_sub_path = 'T24H-X850M-Y880M_2013-01-01_2017-01-01'

array(['T12H-X850M-Y880M_2013-01-01_2017-01-01',
       'T1H-X1700M-Y1760M_2013-01-01_2017-01-01',
       'T24H-X1275M-Y1320M_2012-01-01_2019-01-01',
       'T24H-X1700M-Y1760M_2012-01-01_2019-01-01',
       'T24H-X255M-Y220M_2013-01-01_2017-01-01',
       'T24H-X425M-Y440M_2012-01-01_2019-01-01',
       'T24H-X425M-Y440M_2013-01-01_2017-01-01',
       'T24H-X850M-Y880M_2012-01-01_2019-01-01',
       'T24H-X850M-Y880M_2013-01-01_2017-01-01',
       'T24H-X85M-Y110M_2013-01-01_2017-01-01',
       'T3H-X850M-Y880M_2013-01-01_2017-01-01',
       'T6H-X850M-Y880M_2013-01-01_2017-01-01'], dtype='<U40')


In [3]:
conf, shaper, sparse_crimes, crime_feature_indices = setup(data_sub_path=data_sub_path)

In [4]:
from utils.preprocessing import Shaper

In [5]:
for i,k in enumerate(crime_feature_indices):
    print(f"'{k}':{i},")

'TOTAL':0,
'THEFT':1,
'BATTERY':2,
'CRIMINAL DAMAGE':3,
'NARCOTICS':4,
'ASSAULT':5,
'BURGLARY':6,
'MOTOR VEHICLE THEFT':7,
'ROBBERY':8,
'Arrest':9,


In [6]:
i = 0
crimes = sparse_crimes[:,i:i+1]
print(conf.shaper_threshold) # sum over all time should be above this threshold
print(conf.shaper_top_k) # if larger than 0 we filter out only the top k most active cells of the data grid
new_shaper = Shaper(crimes, conf)

0
-1


In [11]:
dense_crimes = shaper.squeeze(sparse_crimes)
from utils.utils import describe_array

In [19]:
print(describe_array(dense_crimes[:,4]))

____________________________________________________________
{'max': 43.0,
 'mean': 0.084798899185383,
 'min': 0.0,
 'shape': (1461, 772),
 'std': 0.37116642461852944}
____________________________________________________________


In [56]:
dense_crimes[dense_crimes > 0] = 1

In [57]:
c_now = dense_crimes[:-1,1:-1]
c_prev = dense_crimes[1:,1:-1]

In [58]:
c_now_flt = np.reshape(c_now.swapaxes(1,2),(-1,8))
c_prev_flt = np.reshape(c_prev.swapaxes(1,2),(-1,8))

In [59]:
df_now = pd.DataFrame(c_now_flt,columns=crime_feature_indices[1:-1])
df_prev = pd.DataFrame(c_prev_flt,columns=crime_feature_indices[1:-1])

In [60]:
df_now.corr()

,THEFT,BATTERY,CRIMINAL DAMAGE,NARCOTICS,ASSAULT,BURGLARY,MOTOR VEHICLE THEFT,ROBBERY
THEFT,1.000000,0.105854,0.075943,0.068029,0.066417,0.050817,0.047021,0.060383
BATTERY,0.105854,1.000000,0.101798,0.139691,0.104406,0.061343,0.049696,0.079825
CRIMINAL DAMAGE,0.075943,0.101798,1.000000,0.074156,0.061469,0.051847,0.038608,0.047310
NARCOTICS,0.068029,0.139691,0.074156,1.000000,0.086472,0.047265,0.044761,0.069765
ASSAULT,0.066417,0.104406,0.061469,0.086472,1.000000,0.036869,0.031553,0.050848
BURGLARY,0.050817,0.061343,0.051847,0.047265,0.036869,1.000000,0.028230,0.033261
MOTOR VEHICLE THEFT,0.047021,0.049696,0.038608,0.044761,0.031553,0.028230,1.000000,0.029594
ROBBERY,0.060383,0.079825,0.047310,0.069765,0.050848,0.033261,0.029594,1.000000


In [66]:
X = dense_crimes[:-1,:-1].swapaxes(1,2).reshape(-1,9)
y = dense_crimes[1:,0].reshape(-1)
y[y > 0] = 1
from sklearn.feature_selection import mutual_info_classif

In [67]:
mutual_info_classif(X,y)

array([0.07804144, 0.02078052, 0.02245209, 0.01046461, 0.01601758,
       0.0073597 , 0.00517525, 0.00221414, 0.00494581])

# Interactive Plotly Plots

In [1]:
from ipywidgets import widgets

mapbox_access_token = open(".mapbox_token").read()
# mapbox_access_token = "open-street-map"
mapbox_access_token

on_state = ('#f050ae', 8)
off_state = ('#ffab00', 8)

mapbox_styles = {
    "sat": "mapbox://styles/bernsblack/ckecz0wr52pfc1at7tvu43fmj", 
    "mono": "mapbox://styles/bernsblack/ckecyyizy065w19psrikmeo5d",
    "dark": "mapbox://styles/bernsblack/ckeikbchd254619s57np6iyum",
}

In [2]:
from utils.data_processing import time_series_to_time_index

In [3]:
# importing ogirinal data
import pandas as pd
import numpy as np
import plotly.graph_objs as go

In [4]:
color_palette = ["#33a8c7","#52e3e1","#a0e426","#fdf148","#ffab00","#f77976","#f050ae","#d883ff","#9336fd"]

In [5]:
df = pd.read_csv("data/original/Crimes_Chicago_2001_to_2019.csv",nrows=500_000)

In [6]:
df = df[[
    'ID',
    'Date',
    'Primary Type',
    'Arrest',
    'Year',
    'Latitude',
    'Longitude',
    'Census Tracts',
]].dropna()

In [7]:
# filter out rare crime categories
top_k_crimes_cats = len(color_palette)
crime_categories = list(df['Primary Type'].value_counts().index)[:top_k_crimes_cats]
df = df[df['Primary Type'].isin(crime_categories)]

In [8]:
from pandas.api.types import CategoricalDtype
# Label encode the crime catgegories - makes histograms faster
CrimeType = CategoricalDtype(categories=crime_categories, ordered=True)
df['Primary Type'] = df['Primary Type'].astype(CrimeType)
df['c'] = df['Primary Type'].cat.codes

In [9]:
# convert string date to date time - takes the most time
df['Date'] = pd.DatetimeIndex(pd.to_datetime(df['Date']))

In [10]:
df['t'] = time_series_to_time_index(t_series=df.Date, t_step='1D', floor=True)

In [25]:
latlon = df[['Latitude', 'Longitude', 'Primary Type']].sample(10_000)

In [11]:
from utils.utils import ffloor, fceil

In [12]:
from geopy import distance

# get meter per degree estimat
coord_series = df[['Latitude', 'Longitude']]

lat_min, lon_min = coord_series.min()
lat_max, lon_max = coord_series.max()

lat_mean, lon_mean = coord_series.mean()

dy = distance.distance((lat_min, lon_min), (lat_max, lon_min)).m
dx = distance.distance((lat_min, lon_min), (lat_min, lon_max)).m

lat_per_metre = (lat_max - lat_min)/dy
lon_per_metre = (lon_max - lon_min)/dx

ratio_xy = dx/dy
print(f"ratio_xy: {ratio_xy}")
print(f"lat_per_metre: {lat_per_metre}")
print(f"lon_per_metre: {lon_per_metre}")

ratio_xy: 0.8128919785997567
lat_per_metre: 9.003326781922841e-06
lon_per_metre: 1.2003344800028324e-05


### Very important  - all the meta data is mapped from intervals of 0.001 in the lat and lon space with ratios of 8 and 11 to ensure that the grids cels are square

In [13]:
cell_size_m = 400
dlat = cell_size_m*lat_per_metre
dlon = cell_size_m*lon_per_metre
xbins = np.arange(ffloor(lon_min, dlon),fceil(lon_max,dlon), dlon)
ybins = np.arange(ffloor(lat_min, dlat),fceil(lat_max,dlat), dlat)
ny = len(ybins)
nx = len(xbins)

In [24]:
xyt = df[['t', 'c', 'Latitude', 'Longitude']].values
nt = int(np.ceil(df['t'].max()))
nc = len(crime_categories)

tbins = np.arange(0,nt+1,1)
cbins = np.arange(0,nc+1,1)

ybins = np.linspace(lat_min, lat_max, ny)
xbins = np.linspace(lon_min, lon_max, nx)

counts, edges = np.histogramdd(sample=xyt, bins=(tbins, cbins, ybins, xbins)) # (nt, nc, ny, nx))
tbins, cbins, ybins, xbins = edges
counts.shape

(2616, 9, 106, 85)

In [17]:
counts_mean = counts.sum(1).mean(0)

In [21]:
xx, yy = np.meshgrid(xbins, ybins)

grid = go.Scattermapbox(
    lon=xx.flatten(),
    lat=yy.flatten(),
    mode='markers',
    marker_color='red',
)

heatmap = go.Heatmap(
    z=counts_mean,
    x=xbins,
    y=ybins,
    opacity=0.2,
)

go.Figure(
    data=[heatmap], 
    layout_height=800,
#     layout_width=int(800*ratio_xy),
    layout_mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=go.layout.mapbox.Center(
            lat=lat_mean,
            lon=lon_mean,
        ),   
        style=mapbox_styles["sat"],
        pitch=0,
        zoom=11,
    ),          
).show()

In [28]:
# [20,16] - [10,8] - [5,4] - [3,2] - [1,1]
x_scale, y_scale = 10,8
xy_scale = np.array([x_scale, y_scale])  # must be integer so that we can easily sample demographic data
dx, dy = xy_scale * np.array([0.001, 0.001])
meta_info = {}
meta_info["x_scale"] = x_scale
meta_info["y_scale"] = y_scale
meta_info["dx"] = float(dx)
meta_info["dy"] = float(dy)
meta_info["x in metres"] = 85000 * float(dx)
meta_info["y in metres"] = 110000 * float(dy)
meta_info

{'x_scale': 10,
 'y_scale': 8,
 'dx': 0.01,
 'dy': 0.008,
 'x in metres': 850.0,
 'y in metres': 880.0}

In [29]:
import plotly.express as px

fig_height = 800
fig_width = int(ratio_xy*fig_height)

img = counts.sum(1).mean(0)
heatmap = go.Heatmap(z=img,x=xbins, y=ybins, colorscale='viridis', opacity=.4)
scatter = go.Scattergl(
    x=latlon.Longitude,
    y=latlon.Latitude,
    mode='markers',
#     marker_opacity=.8,
    marker_symbol='x-thin',
    marker_color='#ffab00',
#     xaxis='x2',
#     yaxis='y2',
)


n = len(latlon)
mapbox = go.Scattermapbox(
        lat=latlon.Latitude,
        lon=latlon.Longitude,
        mode='markers',
        marker=dict(
            color=[off_state[0]]*n,
            size=[off_state[1]]*n,
            opacity=[0.7]*n,
        ),
        text=None,
)

mapbox_layout=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=go.layout.mapbox.Center(
            lat=lat_mean,
            lon=lon_mean,
        ),
#         style="open-street-map",
#         style=mapbox_styles["mono"],
#         style=mapbox_styles["dark"],    
        style=mapbox_styles["sat"],
        pitch=0,
        zoom=9,
        domain={'x': [0.0, 0.45], 'y': [0, 1]},
)


fig_layout = dict(
    mapbox=mapbox_layout,
    height=fig_height,
    width=fig_width,
#     xaxis=dict(
#         domain=[0, 0.45],
#     ),
#     xaxis2=dict(
#         domain=[0.55, 1.0],
#     ),
#     yaxis2=dict(
#         domain=[0.0, 1.0],
#         anchor="x2",
#     ),
)

f = go.FigureWidget(
#     data=[mapbox],
    data=[heatmap, scatter],
    layout=fig_layout,
)

scatter = f.data[-1]

selection = None
def select_fn(trace, points, selector):
    global selection
    selection = trace, points, selector

for i in range(len(f.data)):
    f.data[i].on_selection(select_fn)
    
# scatter.on_selection(select_fn)

f

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

In [30]:
lat_selected = f.data[1].y[[*f.data[1].selectedpoints]]
lon_selected = f.data[1].x[[*f.data[1].selectedpoints]]
lat_sel_min, lat_sel_max = lat_selected.min(), lat_selected.max()
lon_sel_min, lon_sel_max = lon_selected.min(), lon_selected.max()
lon_sel_min, lon_sel_max

(-87.727018579, -87.61146150200001)

In [31]:
color_map = {k:v for k,v in zip(crime_categories, color_palette)}
def get_color(t):
    return color_map.get(t, '#f050ae')

df['crime_c'] = df['Primary Type'].apply(get_color).to_list()

In [33]:
d = df.query('Year == 2014').sample(n=24000)
lat = d['Latitude']
lon = d['Longitude']
label = d[['Primary Type', 'Date']]
center = d[['Longitude', 'Latitude']].mean()
c = d['crime_c']
n = len(d)
print(len(d))
d.columns.to_list()

24000


['ID',
 'Date',
 'Primary Type',
 'Arrest',
 'Year',
 'Latitude',
 'Longitude',
 'Location',
 'Census Tracts',
 'c',
 't',
 'crime_c']

In [219]:
def get_widget_index(change):
    if isinstance(change, dict) and change['old']:
        old = change['old']
        if isinstance(old, dict) and old['index']:
            return old['index']
    return None

In [305]:
from ipywidgets import Layout

date_range = pd.date_range(df.Date.min().ceil('1D'), df.Date.max().floor('1D'), freq='1D')
date_range_slider = widgets.SelectionRangeSlider(
    options=[d.strftime('%y/%m/%d') for d in date_range],
    index=(0, len(date_range)-1),
    description='Date Range:',
    disabled=False,
    orientation = 'horizontal',
    layout=Layout(width='95%'),
    continuous_update=False,
)


# init_mask = df.ID == df.ID

date_range_filter = None 

def set_date_range_filter(change):
    global date_range_filter
    
    index = get_widget_index(change)
    if index:
        date_range_filter = index
#         date_range_mask = (df.Date > date_range_limits[0]) & (df.Date <= date_range_limits[1])        

date_range_slider.observe(set_date_range_filter)
date_range_slider

SelectionRangeSlider(continuous_update=False, description='Date Range:', index=(0, 2615), layout=Layout(width=…

In [306]:
category_options = ['ALL',*crime_categories]
crime_types_dropdown = widgets.Dropdown(
    options=category_options,
    value='ALL',
    description='Crime Type:',
)


crime_category_filter = None

def set_crime_category_filter(change):
    global crime_category_filter
    
    index = get_widget_index(change)
    if isinstance(index, int):
        category = category_options[index]
        if category == 'ALL':
            crime_category_filter = None
        else:
            crime_category_filter = category
    
crime_types_dropdown.observe(set_crime_category_filter)    
crime_types_dropdown

Dropdown(description='Crime Type:', options=('ALL', 'THEFT', 'BATTERY', 'CRIMINAL DAMAGE', 'NARCOTICS', 'ASSAU…

In [312]:
time_index_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=len(date_range)-1,
    step=1,
    description='Time Index:',
    continuous_update=False,
    layout=Layout(width='80%'),
)

play_button = widgets.Play(
    value=1,
    min=time_index_slider.min,
    max=time_index_slider.max,
    step=1,
    interval=500,
    description="Press play",
    disabled=False
)

widgets.jslink((play_button, 'value'), (time_index_slider, 'value'))

Link(source=(Play(value=1, description='Press play', interval=500, max=2615), 'value'), target=(IntSlider(valu…

In [311]:

widgets.VBox([
    widgets.HBox([crime_types_dropdown, date_range_slider]),
    widgets.HBox([play_button, time_index_slider])    
])

In [280]:
crime_category_filter, date_range_filter

('CRIMINAL DAMAGE', [755, 2615])

In [313]:
df[df.t == 4]

,ID,Date,Primary Type,Arrest,Year,Latitude,Longitude,Location,Census Tracts,c,t,crime_c
1994871,8641238,2012-06-02 16:00:00,CRIMINAL DAMAGE,False,2012,41.742016,-87.562439,"(41.742015833, -87.562438515)",478.0,2,4,#a0e426
1994872,8640779,2012-06-02 16:00:00,BATTERY,True,2012,41.688850,-87.631809,"(41.688850014, -87.631808697)",660.0,1,4,#52e3e1
1994873,8641375,2012-06-02 16:00:00,CRIMINAL DAMAGE,False,2012,41.849218,-87.691805,"(41.849217934, -87.691805133)",263.0,2,4,#a0e426
1994874,8641597,2012-06-02 16:00:00,BATTERY,False,2012,41.852872,-87.630563,"(41.852871631, -87.630562503)",3.0,1,4,#52e3e1
1994875,8646758,2012-06-02 16:00:00,THEFT,False,2012,41.929098,-87.647278,"(41.929097881, -87.647277762)",794.0,0,4,#33a8c7
...,...,...,...,...,...,...,...,...,...,...,...,...
1995872,8639494,2012-06-01 16:10:00,THEFT,True,2012,41.874271,-87.657499,"(41.874271066, -87.657499107)",88.0,0,4,#33a8c7
1995873,8639479,2012-06-01 16:10:00,BATTERY,False,2012,41.750565,-87.649193,"(41.750564801, -87.649193107)",487.0,1,4,#52e3e1
1995874,8639468,2012-06-01 16:06:00,THEFT,True,2012,41.882457,-87.627848,"(41.882457198, -87.627847776)",92.0,0,4,#33a8c7
1995875,8639465,2012-06-01 16:05:00,BATTERY,True,2012,41.796216,-87.723246,"(41.796216157, -87.723245665)",792.0,1,4,#52e3e1


In [269]:
def update_plot():
    global sub
    mask = crime_type_mask & date_range_mask
    sub = df[mask]
    
#     with fig.batch_update():
#         scatter.lon = sub.Longitude
#         scatter.lat = sub.Latitude
# #         scatter.text = text
    

In [262]:
len(df) - len(sub)

1278991

In [286]:
scatter = go.Scattermapbox(
        lat=lat.to_numpy(),
        lon=lon.to_numpy(),
        mode='markers',
        marker=dict(
            color=[off_state[0]]*n,
            size=[off_state[1]]*n,
            opacity=[0.7]*n,
        ),
        text=label,
)


layout = go.Layout(
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=go.layout.mapbox.Center(
            lat=center.Latitude,
            lon=center.Longitude,
        ),
#         style=mapbox_styles["mono"],
#         style=mapbox_styles["dark"],    
        style=mapbox_styles["sat"],
        pitch=0,
        zoom=9,
#         domain={'x': [0.0, 0.3], 'y': [0, 1]},
    ),
#     height=700, 
#     width=1200,
    
#     xaxis=dict(
#         domain=[0, 0.3],
#     ),
#     xaxis2=dict(
#         domain=[0.4, 1.0],
#     ),
#     yaxis2=dict(
#         domain=[0.0, 1.0],
#         anchor="x2",
#     ),
)

fig = go.FigureWidget(data=[scatter], layout=layout)

scatter = fig.data[0]

selected = []
prev_select = []
t = 0
def select_fn(trace, points, selector):
    global t
    c = list(trace.marker.color)
    s = list(trace.marker.size)

    for i in prev_select:
        c[i], s[i] = off_state        

    for i in points.point_inds:
        c[i], s[i] = on_state  
    
#     mid_x = np.mean(points.xs)
#     mid_y = np.mean(points.ys)
    
    with fig.batch_update():
#         f.title = f"Mid point of selected is {(mid_x, mid_y)}"
        trace.marker.color = c
        trace.marker.size = s
        curve.x = np.arange(1000)
        curve.y = np.random.randn(1000)

    selected.clear()
    prev_select.clear()
    selected.append([trace, points, selector])
    prev_select.extend(points.point_inds)
        
    
scatter.on_selection(select_fn)
# scatter.on_hover(select_fn)


crime_types_dropdown = widgets.Dropdown(
    options=['ALL',*crime_categories],
    value='ALL',
    description='Crime Type:',
)

month_slider = widgets.IntSlider(
    value=1.0,
    min=1.0,
    max=12.0,
    step=1.0,
    description='Month:',
    continuous_update=False,
)

chng = 0
def widget_callback(change):
    global chng
    chng = change
    
    selected_crime_type = crime_types_dropdown.value
    if selected_crime_type != 'ALL':
        mask = d["Primary Type"] == selected_crime_type
        filtered = d[mask]
        text = label[mask]
    else:
        filtered = d
        text = label
    
    selected_month = month_slider.value
    if selected_month:
        mask = filtered['Date'].map(lambda x: x.month) == selected_month
        filtered = filtered[mask]
        text = text[mask]
        
    
    with fig.batch_update():
        scatter.lon = filtered.Longitude
        scatter.lat = filtered.Latitude
        scatter.text = text
#     scatter.marker.color = get_color(selected)


crime_types_dropdown.observe(widget_callback, names="value")
month_slider.observe(widget_callback, names="value")


play_button = widgets.Play(
    value=1,
    min=1,
    max=12,
    step=1,
    interval=500,
    description="Press play",
    disabled=False
)

widgets.jslink((play_button, 'value'), (month_slider, 'value'))



container = widgets.HBox([month_slider, crime_types_dropdown, play_button])
widgets.VBox([
    container,
    fig,
])

ValueError: 
Invalid property path 'mapbox._derived' for layout


ValueError: 
Invalid property path 'mapbox._derived' for layout


ValueError: 
Invalid property path 'mapbox._derived' for layout


ValueError: 
Invalid property path 'mapbox._derived' for layout


ValueError: 
Invalid property path 'mapbox._derived' for layout


...

ValueError: 
Invalid property path 'mapbox._derived' for layout


ValueError: 
Invalid property path 'mapbox._derived' for layout


ValueError: 
Invalid property path 'mapbox._derived' for layout


ValueError: 
Invalid property path 'mapbox._derived' for layout


ValueError: 
Invalid property path 'mapbox._derived' for layout


In [289]:
fig.layout

Layout({
    'autosize': True,
    'hovermode': 'closest',
    'mapbox': {'accesstoken': ('pk.eyJ1IjoiYmVybnNibGFjayIsImE' ... 'DM4In0.dpYqqUJsBxuc6DBqTsHFJw\n'),
               'bearing': 0,
               'center': {'lat': 41.841179426188006, 'lon': -87.67220241239322},
               'pitch': 0,
               'style': 'mapbox://styles/bernsblack/ckecz0wr52pfc1at7tvu43fmj',
               'zoom': 9},
    'template': '...'
})